<a href="https://colab.research.google.com/github/ba18406/Lab001/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", delimiter=',', header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  rated = np.where(arr!=99)
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [4]:
arr = df.values
validationData, idx = replace(arr)
print(validationData)

[[ 74.    -7.82   8.79 ...  99.    99.    99.  ]
 [100.     4.08  -0.29 ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 99.    99.    99.   ...  99.    99.    99.  ]
 [ 99.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]


In [5]:
n_latent_factors = 5
user_ratings = df.values

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(latent_user_preferences.shape)
print(latent_item_features.shape)

(24983, 5)
(101, 5)


In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.00001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [0]:
sgd()

1180.667536351486


In [8]:
user_ratings = validationData

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[1.2468455 , 0.92147787, 1.33541321, ..., 1.16396852, 1.08955175,
        0.69589129],
       [1.4411739 , 0.99593929, 1.25391909, ..., 1.03033589, 1.16187048,
        0.83002742],
       [1.90131085, 1.33454711, 1.93655116, ..., 2.01637291, 2.0330576 ,
        1.28639833],
       ...,
       [0.94033909, 0.90416103, 1.09800527, ..., 0.80850728, 0.74586757,
        0.28368989],
       [1.54859497, 1.16680756, 1.67030536, ..., 1.63679387, 1.6208125 ,
        0.93582011],
       [1.86660656, 1.33764511, 1.75874264, ..., 1.52186116, 1.33535471,
        1.10262158]])

In [10]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 1.246845498382529)","(-7.82, 0.9214778681418463)","(8.79, 1.3354132113383421)","(-9.66, 0.586620622520126)","(-8.16, 1.1522783240569268)","(-7.52, 1.1332966334305166)","(-8.5, 1.0152484328887996)","(-9.85, 0.9742276308868487)","(4.17, 1.0464926383730568)","(-8.98, 0.6687769012505113)","(-4.76, 0.874819294617725)","(99.0, 0.9531671872527909)","(-6.75, 0.7844765158412147)","(99.0, 0.5854768074108907)","(8.45, 1.373544859984133)","(99.0, 0.5937332255872777)","(-7.52, 0.8944415991987201)","(-7.43, 0.9851505657751238)","(-9.81, 0.9726502664756373)","(-9.85, 0.8548431430760127)","(-9.85, 1.2100652934815248)","(-9.37, 0.7821993995547243)","(99.0, 0.6622040566690734)","(-4.37, 0.5464071706927286)","(-9.81, 0.5202305570418989)","(-8.5, 1.127760092327149)","(1.12, 1.1060321224072265)","(7.82, 0.9683897414866731)","(2.86, 0.9383857392487951)","(9.13, 0.7400896873473356)","(-7.43, 0.7252343719498802)","(99.0, 1.0808937462112784)","(99.0, 0.9887658073294331)","(-9.08, 0.8501088430551202)","(7.82, 0.8118547044309674)","(99.0, 1.1325822952533662)","(4.95, 0.8680478325671422)","(-9.17, 0.9744119367891887)","(-8.4, 0.7918041874848619)","(-8.4, 0.614645391045043)",...,"(8.59, 0.8422956649532822)","(3.59, 1.0367876588069542)","(-6.84, 0.9402917016322826)","(-9.03, 1.0379548640639857)","(2.82, 0.8354919294363119)","(-1.36, 0.8747922787929374)","(-9.08, 0.7420160747427478)","(8.3, 0.8775834977155884)","(5.68, 0.5381416316720644)","(-4.81, 0.8594535655535365)","(99.0, 0.6898588065246455)","(99.0, 0.6993552729263982)","(99.0, 0.8949635584043526)","(99.0, 0.3582307956227171)","(99.0, 0.9243804004970668)","(99.0, 0.9469962729568844)","(99.0, 1.262055530663901)","(-9.42, 0.817111337618206)","(99.0, 1.1507778286773556)","(99.0, 0.8598013247090082)","(99.0, 0.8918491759299385)","(-7.72, 0.7243925754000926)","(99.0, 0.9267821243952689)","(99.0, 0.9889535162632381)","(99.0, 0.8713479830618197)","(99.0, 1.2455418604549602)","(99.0, 0.7637246535916551)","(99.0, 0.7868422452201272)","(99.0, 0.9890815552752455)","(99.0, 1.2234837239787808)","(2.82, 0.8192285523679884)","(99.0, 1.0517024543507199)","(99.0, 0.9334418686856665)","(99.0, 0.7199305159149901)","(99.0, 0.7989260101980753)","(99.0, 0.7708608470214572)","(-5.63, 1.0163564796840245)","(99.0, 1.1639685167865028)","(99.0, 1.0895517477643692)","(99.0, 0.6958912888379641)"
1,"(100.0, 1.4411738990622265)","(4.08, 0.9959392933596096)","(-0.29, 1.2539190927668171)","(6.36, 0.8436247523108547)","(4.37, 1.1773478792489156)","(-2.38, 1.075300653965221)","(-9.66, 0.64953789310533)","(-0.73, 1.0703806609887954)","(-5.34, 0.7069977804209745)","(99.0, 1.0172049482060816)","(9.22, 0.6259450161849691)","(6.75, 1.011071475490578)","(99.0, 0.7172047959807096)","(99.0, 0.49310251630917806)","(7.43, 1.296389683607128)","(4.56, 0.6459795118681079)","(-0.97, 0.9093577940531085)","(4.66, 0.7114576027620424)","(-0.68, 1.0456758984166294)","(3.3, 1.0846159719408124)","(-1.21, 1.0458979706045115)","(0.87, 0.9297252093438075)","(8.64, 0.7932943906099528)","(99.0, 0.5482416578716807)","(9.17, 0.5221224102875947)","(0.05, 1.266025511523667)","(99.0, 1.354191470454395)","(4.71, 0.5441904116297007)","(99.0, 0.706985569955061)","(-0.39, 0.6996042125972253)","(6.99, 0.9777358251444167)","(6.5, 1.0825170276442821)","(-0.92, 0.8874749245461635)","(7.14, 0.5607255312718968)","(9.03, 0.4394667669995616)","(99.0, 1.1966374696833817)","(0.73, 0.8603127568745733)","(7.09, 1.0308369737207248)","(3.4, 0.6463323796718197)","(-0.87, 0.7061665310934473)",...,"(-6.7, 0.6516456741795591)","(-3.35, 1.1281068507420466)","(-9.03, 1.1219728942761567)","(4.47, 0.6484680697172985)","(4.08, 0.9156926180355606)","(-3.83, 0.8080789910556168)","(8.74, 0.808106932846746)","(1.12, 0.7327093679234759)","(0.78, 0.68026953